<h1 style='text-align: center; text-decoration: underline;'>argentum</h1>
<p style='text-align: center;'>
    Macroeconomic Modeling of Currency Debasement in the Roman Empire
</p>

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np

## Data Construction

## Linear Regression Analysis